In [1]:
#Import Libraries
import pandas as pd
import string
import nltk
from nltk import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import math
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandas import Series, DataFrame
from matplotlib import rcParams
import pandas as pd
from collections import Counter
from itertools import chain
#from wordcloud import WordCloud, STOPWORDS
import re
import warnings
warnings.filterwarnings("ignore")
import collections
import os
from matplotlib import pyplot as plt
import matplotlib.colors as mcolors
rcParams['figure.figsize'] = 20,12
ps = PorterStemmer()
nltk.download('punkt')
nltk.download('stopwords')
stop = stopwords.words('english')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vasil\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vasil\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [15]:
#load the data
reviews = pd.read_csv(r"reviews_all_preprocessed.csv")
stop1 = ['food','good','nice','price','place','servic','restaur','visit','friendli','thessaloniki','order','great','dish','recommend','amaz','one','realli','us','serv','staff','qualiti','’','time', 'tri', 'experi', 'come', 'small', 'local', 'even', 'excel', 'reason', 'atmospher', 'best', 'love', 'enjoy', 'would', 'greek', 'well', 'also', 'fresh', 'back', 'like', 'tast', 'wine', 'definit', 'tabl', 'offer', 'menu', 'salad', 'drink', 'waiter', 'tradit', 'tasti', 'delici', 'day', 'peopl', 'go', 'meal', 'fish', 'dinner', 'eat', 'cook', 'portion', 'look', 'meat', 'citi', 'everyth', 'music', 'locat', 'lunch','town', 'warm', 'year', 'outsid', 'street', 'chicken', 'coffe', 'night', 'perfect', 'much', 'want', 'view', 'got', 'plate', 'littl', 'sea', 'expect', 'owner', 'ok', 'make', 'way', 'next', 'sit', 'wait', 'pizza', 'friend', 'cuisin', 'high', 'lot', 'differ', 'kind', 'noth', 'special', 'choic', 'decor', 'live', 'greec', 'alway', 'center', 'better', 'bread', 'ask', 'help', 'dont', 'get', 'mani', 'top', 'fantast', 'thing', 'say', 'two', 'first', 'didnt', 'busi', 'went', 'came', 'cours', 'though', 'polit', 'find', 'fast', 'end', 'grill', 'take', 'found', 'ever', 'quit', 'euro', 'wonder', 'everi', 'starter', 'bar', 'area', 'main', '2', 'full', 'hous', 'highli', 'could', 'burger', 'chees', 'made', 'big', 'valu', 'must', 'select', 'smoke', 'beauti', 'last', 'around', 'someth', 'varieti', 'money', 'feel', 'fri', 'beer', 'seafood', 'bit', 'insid', 'sure', 'worth', 'expens', 'disappoint', 'need', 'ate', 'famili', 'dessert','\"','”','“']

# Topic Modeling

In [23]:
#Create a new dataframe that contains the reviews and remove the extra stopwords
reviews_lda = reviews.copy()
#reviews_lda['Review'] = reviews_lda['Review'].apply(lambda x: [item for item in x if item not in stop1])

#Convert "Rating_date" into datetime and sort reviews based on that
reviews_lda['Rating_date'] = pd.to_datetime(reviews_lda['Rating_date'])
reviews_lda.sort_values(by = 'Rating_date', inplace = True)

#Keep only the columns that we need for LDA (Rating_date and Review)
sorted_reviews = reviews_lda.drop(reviews.columns.difference(['Rating_date', 'Review']), 1)
sorted_reviews.reset_index(drop = True, inplace = True)

#Convert each review into a list of the tokens it includes
temp_uni_sorted = []
for i in range(len(sorted_reviews)):
    temp_uni_sorted.append(sorted_reviews['Review'][i])

In [25]:
#Create chronologically ordered groups, where each one contains 2000 reviews
n = 2000
split_accross_years = [temp_uni_sorted[i:i + n] for i in range(0, len(temp_uni_sorted), n)]

In [7]:
import gensim
from gensim.utils import simple_preprocess
import nltk
import gensim.corpora as corpora    
from pprint import pprint

# LDA for each group of reviews in chronological order
for i in split_accross_years:
    
    # Create Dictionary
    id2word = corpora.Dictionary(i)
    # Create Corpus
    texts = i
    # Term Document Frequency
    corpus = [id2word.doc2bow(text) for text in texts]# View

    # Define the number of topics
    num_topics = 4    
    # Build LDA model
    lda_model = gensim.models.LdaMulticore(corpus = corpus,
                                           id2word = id2word,
                                           num_topics = num_topics)
    
    #Print the topics and the most important keywords
    #pprint(lda_model.print_topics())
    doc_lda = lda_model[corpus]
    
    cols = [color for name, color in mcolors.TABLEAU_COLORS.items()] 
    
    cloud = WordCloud(background_color = 'white',
                      width = 2500,
                      height = 1800,
                      max_words = 15,
                      colormap = 'tab10',
                      color_func = lambda *args, **kwargs: cols[i],
                      prefer_horizontal = 1.0)
    
    topics = lda_model.show_topics(formatted = False)
    
    fig, axes = plt.subplots(1, 4, figsize = (8,8), sharex = True, sharey = True)
    
    for i, ax in enumerate(axes.flatten()):
        fig.add_subplot(ax)
        topic_words = dict(topics[i][1])
        cloud.generate_from_frequencies(topic_words, max_font_size=300)
        plt.gca().imshow(cloud)
        plt.gca().set_title('Topic ' + str(i), fontdict=dict(size=16))
        plt.gca().axis('off')
    
    
    plt.subplots_adjust(wspace=0, hspace=0)
    plt.axis('off')
    plt.margins(x=0, y=0)
    plt.tight_layout()

## Fastest Growing and Fastest Shrinking Words

In [26]:
reviews['Rating_date'] =pd.to_datetime(reviews_lda['Rating_date'])
#we sort the dataframe by time so each chuck represents a time splot 
reviews.sort_values(by='Rating_date',inplace=True)
test = reviews.copy()
#n represents the size of each chunk we want to cut the dataframe
n = 1597
#x represents the array that consists of dictionaries on each cell and its size is the amount of chunks we have
x = []
#the loop stops when the dataframe is empty
while len(test)>0:
    #we just take the first n rows and then we proceed removing those rows after we have processed them
    temp = test.head(n)
    test = test.iloc[n: , :]
    temp_uni = []
    for i in range(len(temp)):
        temp_uni += temp['Review'].iloc[i]
    temp_uni = pd.Series(temp_uni)
#we create a dictionary for each chuck that has the shape of word:frequency
    x.append(temp_uni.value_counts().to_dict())

In [27]:
temp_uni = []
for i in range(len(reviews)):
    temp_uni += reviews['Review'].iloc[i]
uni_words = {}

#we find all the words and create a dictionary with them
for i in temp_uni:
    uni_words.update({i:0})

In [28]:
#word dictionary represents the for each worth the growth of it
word = {}
#dont_count dictionary exists so we can not count the words that dont exist in one or more time slots.
dont_count = {}
for i in range(len(x)-1):
    for z in uni_words.keys():
        try:
            next_time = x[i+1][z]
            try:
                current_time = x[i][z]
#here we calculate the growth of the specific time slot
                temp = (next_time-current_time)/current_time
            except:
                temp = 1
                dont_count.update({z:1})
        except:
            temp = -1
            #dont_count.update({x[i+1][z]:1})
            dont_count.update({z:1})
        try:
            word.update({z:temp+word[z]})
        except:
            word.update({z:temp})

In [29]:
#we transofrm the word dictionary in a dataframe and then we drop each word that exists in the dont_count dictionary
Growth = pd.DataFrame.from_dict([word]).T
for i in dont_count.keys():
    Growth.drop(str(i),inplace = True)

#here we get the top 10 and bottom 10 words that represent the positive and negative growth repsectively
Top = Growth.sort_values(by = 0,ascending =False).head(12)
Bottom = Growth.sort_values(by = 0,ascending =False).tail(10)
stop2 = ['‘','nowher']
Top.reset_index(inplace=True)
Top = Top[Top["index"].str.contains('‘')==False]
Top = Top[Top["index"].str.contains('nowher')==False]
Top.set_index('index', inplace=True)

In [ ]:
word = []
for i in range(len(x)):
        word.append(x[i]['aristotel'])

In [ ]:
import matplotlib.pyplot as plt

plt.plot(word)
plt.show()

# Map of Locations

In [34]:
from pycountry_convert import country_alpha2_to_continent_code, country_name_to_country_alpha2
from geopy.geocoders import Nominatim

In [35]:
#get county from city name
def get_country_from_city_name(x):
    try:
        geolocator = Nominatim(user_agent = "geoapiExercises")
        location = geolocator.geocode(x)
        address = location.raw
        return address.get('display_name').split(',')[-1]
    except:
        return 'Unknown Country'

In [36]:
#get country from the city name
def get_countries(x):
    try:
        array = x.split('from')[1]
    except:
        return 'no country given'
    try:
        return get_country_from_city_name(array.split(',')[1])
    except:
        try:
            return get_country_from_city_name(array)
        except:
            return 'Miss_spelled_City_Name'

In [37]:
#functions to get longitude and latitude data from country name
def geolocate_latitude(country):
        geolocator = Nominatim(user_agent = "geoapiExercises")
        loc = geolocator.geocode(country)
        return (loc.latitude)
    
def geolocate_longitude(country):
        geolocator = Nominatim(user_agent = "geoapiExercises")
        loc = geolocator.geocode(country)
        return (loc.longitude)

In [38]:
testdf = reviews.copy()

In [ ]:
testdf['Country'] = testdf['Review_Distribution'].apply(lambda x: get_countries(x))
testdf['Country'] = testdf['Country'].str.strip()
testdf['Country'] = testdf['Country'].apply(lambda x: get_country_from_city_name(x))
# The above cell takes a while to run so we will present you how the dataframe will be transformed below.

In [ ]:
reviews_map = pd.read_csv(r"beforegetcontinent.csv")
reviews_map.drop(columns = ["Unnamed: 0"], inplace = True)
reviews_map

In [ ]:
testdf = testdf[testdf['Country'] != 'Unknown Country']
testdf.reset_index(drop = True,inplace = True)

In [ ]:
testdf['Latitude'] = testdf['Country'].apply(lambda x: geolocate_latitude(x))
testdf['Longitude'] = testdf['Country'].apply(lambda x: geolocate_longitude(x))
# The above cell takes a while to run so we will present you how the dataframe will be transformed below.

In [ ]:
reviews_map = pd.read_csv(r"latlon.csv")
reviews_map.drop(columns = ["Unnamed: 0"], inplace = True)
reviews_map.drop(columns = ["Unnamed: 0.1"], inplace = True)
reviews_map

In [ ]:
testdf = reviews_map.copy()

In [ ]:
# Create a world map to show distributions of users 
import folium
from folium.plugins import MarkerCluster
#empty map
world_map= folium.Map(tiles="cartodbpositron")
marker_cluster = MarkerCluster().add_to(world_map)
for i in range(len(testdf)):
        lat = testdf.iloc[i]['Latitude']
        long = testdf.iloc[i]['Longitude']
        radius=5
        folium.CircleMarker(location = [lat, long], radius=radius, fill =True).add_to(marker_cluster)
#show the map
world_map